# Logistic regression

In [1]:
%load_ext sql

/Users/fmcquillan/anaconda/lib/python2.7/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/Users/fmcquillan/anaconda/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [2]:
# Greenplum 4.3.10.0
# %sql postgresql://gpdbchina@10.194.10.68:61000/madlib
        
# PostgreSQL local
%sql postgresql://fmcquillan@localhost:5432/madlib

u'Connected: fmcquillan@madlib'

In [3]:
%sql select madlib.version();
#%sql select version();

1 rows affected.


version
"MADlib version: 1.12, git revision: unknown, cmake configuration time: Wed Aug 23 23:07:18 UTC 2017, build type: Release, build system: Darwin-16.7.0, C compiler: Clang, C++ compiler: Clang"


# 1. Load data
This data set is related to predicting a second heart attack given treatment and health factors.

In [30]:
%%sql
DROP TABLE IF EXISTS patients;

CREATE TABLE patients( id INTEGER NOT NULL,
                       second_attack INTEGER,
                       treatment INTEGER,
                       trait_anxiety INTEGER);
                         
INSERT INTO patients VALUES 
(1,  1, 1, 70),
(2,  1, 1, 80),
(3,  1, 1, 50),
(4,  1, 0, 60),
(5,  1, 0, 40),
(6,  1, 0, 65),
(7,  1, 0, 75),
(8,  1, 0, 80),
(9,  1, 0, 70),
(10, 1, 0, 60),
(11, 0, 1, 65),
(12, 0, 1, 50),
(13, 0, 1, 45),
(14, 0, 1, 35),
(15, 0, 1, 40),
(16, 0, 1, 50),
(17, 0, 0, 55),
(18, 0, 0, 45),
(19, 0, 0, 50),
(20, 0, 0, 60);

SELECT * FROM patients ORDER BY id;

Done.
Done.
20 rows affected.
20 rows affected.


id,second_attack,treatment,trait_anxiety
1,1,1,70
2,1,1,80
3,1,1,50
4,1,0,60
5,1,0,40
6,1,0,65
7,1,0,75
8,1,0,80
9,1,0,70
10,1,0,60


# 2.  Train regression model

In [31]:
%%sql
DROP TABLE IF EXISTS patients_logregr, patients_logregr_summary;

SELECT madlib.logregr_train( 'patients',                             -- Source table
                             'patients_logregr',                     -- Output table
                             'second_attack',                        -- Dependent variable
                             'ARRAY[1, treatment, trait_anxiety]',   -- Feature vector
                             NULL,                                   -- Grouping
                             20,                                     -- Max iterations
                             'irls'                                  -- Optimizer to use
                           );

SELECT * FROM patients_logregr_summary;

Done.
1 rows affected.
1 rows affected.


method,source_table,out_table,dependent_varname,independent_varname,optimizer_params,num_all_groups,num_failed_groups,num_rows_processed,num_missing_rows_skipped,grouping_col
logregr,patients,patients_logregr,second_attack,"ARRAY[1, treatment, trait_anxiety]","optimizer=irls, max_iter=20, tolerance=0.0001",1,0,20,0,None


# 3.  View regression results

In [15]:
%%sql
SELECT * from patients_logregr;

1 rows affected.


coef,log_likelihood,std_err,z_stats,p_values,odds_ratios,condition_no,num_rows_processed,num_missing_rows_skipped,num_iterations,variance_covariance
"[-6.36346994178192, -1.02410605239327, 0.119044916668607]",-9.41018298389,"[3.21389766375099, 1.17107844860319, 0.0549790458269317]","[-1.97998524145757, -0.874498248699539, 2.16527796868916]","[0.0477051870698145, 0.381846973530455, 0.0303664045046183]","[0.00172337630923221, 0.359117354054956, 1.12642051220895]",326.081922792,20,0,5,"[[10.329138193064, -0.474304665195738, -0.171995901260057], [-0.474304665195738, 1.37142473278286, -0.00119520703381591], [-0.171995901260057, -0.00119520703381591, 0.00302269548003986]]"


# 4. Output formatting
Alternatively, unnest the arrays in the results for easier reading of output:

In [13]:
%%sql
SELECT unnest(array['intercept', 'treatment', 'trait_anxiety']) as attribute,
       unnest(coef) as coefficient,
       unnest(std_err) as standard_error,
       unnest(z_stats) as z_stat,
       unnest(p_values) as pvalue,
       unnest(odds_ratios) as odds_ratio
    FROM patients_logregr;

3 rows affected.


attribute,coefficient,standard_error,z_stat,pvalue,odds_ratio
intercept,-6.36346994178,3.21389766375,-1.97998524146,0.0477051870698,0.00172337630923
treatment,-1.02410605239,1.1710784486,-0.8744982487,0.38184697353,0.359117354055
trait_anxiety,0.119044916669,0.0549790458269,2.16527796869,0.0303664045046,1.12642051221


# 5. Predict outcome
This example uses the original data table to perform the prediction. Typically a different test dataset with the same features as the original training dataset would be used for prediction.

In [32]:
%%sql
-- Display prediction value along with the original value
SELECT p.id, madlib.logregr_predict(coef, ARRAY[1, treatment, trait_anxiety]),
       p.second_attack::BOOLEAN
FROM patients p, patients_logregr m
ORDER BY p.id;

20 rows affected.


id,logregr_predict,second_attack
1,True,True
2,True,True
3,False,True
4,True,True
5,False,True
6,True,True
7,True,True
8,True,True
9,True,True
10,True,True


# 6. Predict probability
Predicting the probability of the dependent variable being TRUE.

In [33]:
%%sql
-- Display prediction value along with the original value
SELECT p.id, madlib.logregr_predict_prob(coef, ARRAY[1, treatment, trait_anxiety]),
       p.second_attack::BOOLEAN
FROM patients p, patients_logregr m
ORDER BY p.id;

20 rows affected.


id,logregr_predict_prob,second_attack
1,0.720223028942,True
2,0.894354902502,True
3,0.192269541755,True
4,0.685513072239,True
5,0.167747881509,True
6,0.798098108915,True
7,0.928568075753,True
8,0.959305763694,True
9,0.877576117431,True
10,0.685513072239,True
